# 딥러닝 기초

## CH5 과제

## 2020100381 안정빈

# CH5. 오차역전파법
+ 수치 미분을 사용한 가중치 매개변수의 기울기를 구하는 방법은 계산 시간이 오래 걸리는 단점이 존재
+ 오차역전파법을 이용하면 가중치 매개변수의 기울기를 효율적으로 계산 가능

## 5-1. 계산 그래프

+ 계산 과정을 표현한 그래프로, 복수의 노드와 에지(노드 사이의 직선)으로 표현한다.
+ 노드는 원(O)으로 표현하고, 원 안에 연산 내용을 적는다.
+ 계산 결과를 화살표 위에 적어, 각 노드의 계산 결과가 왼쪽에서 오른쪽으로 전해지게 한다.
+ 계산을 왼쪽에서 오른쪽으로 진행하는 단계를 순전파라고 한다.
+ 계산 그래프는, 국소적 계산을 전파한다. 즉, 각 노드는 자신과 관련한 계산 외에는 다른 계산을 신경쓰지 않는다.
+ 이러한 국소적 계산을 통해 결과를 전달함으로써, 전체를 구성하는 복잡한 계산을 할 수 있다.

### 5-1-3. 계산 그래프의 장점
1. 국소적 계산을 통해, 복잡한 문제를 단순화
2. 중간 계산 결과를 모두 보관 가능
3. 역전파를 통해 '미분'을 효율적으로 계산할 수 있다.

=> 순전파와 역전파를 활용해 각 변수의 미분을 효율적으로 구할 수 있음.

## 5-2. 연쇄법칙
+ 역전파는 '국소적인 미분'을 오른쪽에서 왼쪽으로 전달. 이러한 '국소적 미분'을 전달하는 원리는 연쇄법칙을 따름
+ 연쇄법칙 원리: 합성 함수의 미분은 합성 함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다.

## 5-3. 역전파
+ 덧셈 노드의 역전파: 덧셈 노드는 입력값을 단순히 더하는 역할만 하기 때문에 역전파에서 입력 신호는 절대로 바뀌지 않는다.
  즉, 덧셈 노드는 "입력 신호가 그대로 다음 노드로 전파된다.
+ 곱셈 노드의 역전파: 상류의 값에, 순전파 때의 입력 신호들을 서로 바꾼 값을 곱해서 하류로 보낸다.
+ 덧셈의 역전파에서는 순방향 입력 신호의 값이 필요하지 않지만, 곱셈의 역전파는 순방향 입력 신호의 값이 필요

=> 곱셈 노드 구현시, 순전파의 입력 신호를 변수에 저장한다.

## 5-4. 단순한 계층 구현하기 
사과 쇼핑의 예를 파이썬으로 구현 


### 5-4-1. 곱셈 계층

In [19]:
class MulLayer: # 계산 그래프의 곱셈 노드 
    def __init__(self): # x와 y의 초기화 역할
        self.x=None
        self.y=None
        
    def forward(self, x, y):
        self.x=x
        self.y=y
        out= x*y

        return out 

    def backward(self, dout):
        dx=dout*self.y # x와 y를 바꾼다.
        dy=dout*self.x 

        return dx, dy

In [20]:
apple= 100
apple_num=2
tax= 1.1 

# 계층 구현 
mul_apple_layer=MulLayer()
mul_tax_layer=MulLayer()

# 순전파
apple_price=mul_apple_layer.forward(apple, apple_num)
price=mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [21]:
# 역전파 
dprice=1
dapple_price, dtax=mul_tax_layer.backward(dprice)
dapple, dapple_num=mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


### 5-4-2. 덧셈 계층

In [22]:
class AddLayer:
    def __init__(self):
        pass
    def forward(self, x,y):
        out=x+y
        return out 

    def backward(self, dout):
        dx= dout * 1
        dy= dout * 1
        return dx, dy

In [23]:
apple=100
apple_num=2
orange=150
orange_num=3
tax=1.1

# 계층 
mul_apple_layer=MulLayer()
mul_orange_layer=MulLayer()
add_apple_orange_layer=AddLayer()
mul_tax_layer=MulLayer()

# 순전파
apple_price=mul_apple_layer.forward(apple, apple_num)
orange_price=mul_orange_layer.forward(orange, orange_num)
all_price=add_apple_orange_layer.forward(apple_price, orange_price)
price=mul_tax_layer.forward(all_price, tax)

# 역전파 
dprice=1
dall_price, dtax=mul_tax_layer.backward(dprice)
dapple_price, dorange_price=add_apple_orange_layer.backward(dall_price)
dorange, dorange_num=mul_orange_layer.backward(dorange_price)
dapple, dapple_num= mul_apple_layer.backward(dapple_price)

print(price)
print(dapple_num, dapple, dorange_num, dorange, dtax)

715.0000000000001
110.00000000000001 2.2 165.0 3.3000000000000003 650


## 5-5. 활성화 함수 계층 구현하기


### 5-5-1. RelU 계층
+ ReLU: 순전파 때 입력인 x가 0보다 크면, 역전파는 상류의 값을 그대로 하류로 보냄
  순전파 때 x가 0이하면, 역전파 때는 0을 보냄.

In [24]:
class RelU:
    def __init__(self):
        self.mask= None 

    def forward(self, x):
        self.mask = (x<=0)
        out=x.copy()
        out[self.mask]=0

        return out 

    def backward(self, dout):
        dout[self.mask]=0
        dx=dout

        return dx


mask: True/False로 구성된 넘파이 배열로, ReLU의 역전파를 위한 기억 장치이다.

순전파의 입력인 x의 원소 값이 0이하인 인덱스는 True, 그 외는 False로 유지한다.

역잔파 경우, 순전파 때 만들어둔 mask를 써서 mask의 원소가 True인 곳에는 상류에서 전파된 dout을 0으로 설정한다.

In [25]:
import numpy as np
x=np.array([[1.0, -0.5],[-2.0,3.0]])
print(x)
mask=(x<=0)
print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


### 5-5-2. Sigmoid 계층
+ sigmoid 계층의 역전파는 순전파의 출력(y)만으로 계산이 가능하다.

In [26]:
class Sigmoid:
    def __init__(self):
        self.out=None

    def forwar(self, x):
        out= 1 / (1+np.exp(-x))
        self.out=out
        return out 

    def backward(self, dout):
        dx=dout*(1.0-self.out) * self.out
        return dx

## 5-6. Affine/Softmax 계층 구현하기

### 5-6-1. Affine 계층
+ 입력에 가중치를 더하고, 편향을 더하는 연산
+ 행렬 곱의 역전파는 행렬의 대응하는 차원의 원소 수가 일치하도록 곱을 취하여 구할 수 있다.

In [27]:
# 아핀 계층 구현
X=np.random.rand(2)
W=np.random.rand(2,3)
B=np.random.rand(3)

print(X.shape)
print(W.shape)
print(B.shape)
y=np.dot(X,W)+B

(2,)
(2, 3)
(3,)


### 5-6-2. 배치용 Affine 계층

In [28]:
X_dot_W=np.array([[0,0,0],[10,10,10]])
B=np.array([1,2,3])

print(X_dot_W)
print(X_dot_W+B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


순전파의 편향 덧셈은 각각의 데이터(1 번째 데이터, 2번째 데이터)에 더해진다.

In [29]:
# 역전파
dY=np.array([[1,2,3],[4,5,6]])
print(dY)
dB=np.sum(dY, axis=0) # 편향의 역전파는 그 두 데이터에 대한 미분을 데이터마다 더해서 구함 => 0번째 축에서의 총합과 동일
print(dB)

[[1 2 3]
 [4 5 6]]
[5 7 9]


In [30]:
class Affine:
    def __init__(self, W,b):
        self.W=W
        self.b=b
        self.x=None
        self.dW=None
        self.db=None

    def forward(self, x):
        self.x=x
        out=np.dot(x, self.W) + self.b

        return out 

    def backward(self, dout):
        dx=np.dot(dout, self.W.T)
        self.dW=np.dot(self.x.T, dout)
        self.db=np.sum(dout, axis=0)

        return dx

### 5-6-3. Softmax-with-Loss 계층
+ Affine 계층과 RelU 계층을 통과하며 변환되고, 마지막 softmax 계층에 의해서 입력이 정규화된다.
+ Softmax 계층은 입력 값을 정규화(출력의 합이 1이 되도록 변형)한다.
+ 오차 계층은 softmax의 출력(y)와 정답 레이블(t)를 받고, 이 데이터들로부터 손실 L을 출력한다.
+ softmax 계층의 역전파는 (y1-t1 , y2-t2, y3-t3)과 같이 깔금하게 나온다. -> 즉 softmax 계층의 출력과 정답 레이블의 오차를 있는 그대로 드러내,
   softmax 계층의 앞 계층들은 오차로부터 큰 깨달음을 얻게 된다.
+ 소프트맥스 함수와 크로스 엔트로피 오차를 함께 이용시, 미분 값을 깔끔하게 얻을 수 있고, 효율적인 역전파가 가능해진다.

In [31]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y=None
        self.t=None

    def forward(self, x, t):
        self.t = t
        self.y=softmax(x)
        self.loss=cross_entropy_error(self.y , self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size=self.t.shape[0]
        dx=(self.y - self.t) / batch_size

        return dx

## 5-7. 오차역전파법 구현하기

In [32]:
# 오차역전파법을 적용한 신경망 구현
import sys, os
sys.path.append(os.path.join(os.getcwd(), 'common'))
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size,
                 weight_init_std=0.01):
        # 가중치 초기화
        self.params={}
        self.params['W1']=weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1']=np.zeros(hidden_size)
        self.params['W2']=weight_init_std * np.random.randn(hidden_size,output_size)
        self.params['b2']=np.zeros(output_size)

        # 계층 생성
        self.layers=OrderedDict() # 순서가 있는 딕셔너리 => 추가한 순서를 기억
        self.layers['Affine1']=Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1']=Relu()
        self.layers['Affine2']=Affine(self.params['W2'],self.params['b2'])

        self.lastLayer=SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x=layer.forward(x)

        return x
        
    # x: 입력 데이터, t:정답 레이블 
    def loss(self, x, t):
        y=self.predict(x)

        return self.lastLayer.forward(y,t)

    def accuracy(self, x, t):
        y=self.predict(x)
        y=np.argmax(y, axis=1)
        if t.ndim != 1: t=np.argmax(t, axis=1)

        accuracy=np.sum(y==t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W:self.loss(x,t)

        grads={}
        grads['W1']=numerical_gradient(loss_W, self.params['W1'])
        grads['b1']=numerical_gradient(loss_W, self.params['b1'])
        grads['W2']=numerical_gradient(loss_W, self.params['W2'])
        grads['b2']=numerical_gradient(loss_W, self.params['b2'])

        return grads 

    def gradient(self, x, t):
        # 순전파 
        self.loss(x, t)

        # 역전파
        dout=1
        dout=self.lastLayer.backward(dout)

        layers=list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout=layer.backward(dout)

        grads={}
        grads['W1']=self.layers['Affine1'].dW
        grads['b1']=self.layers['Affine1'].db
        grads['W2']=self.layers['Affine2'].dW
        grads['b2']=self.layers['Affine2'].db

        return grads

### 5-7-3. 오차역전파법으로 구한 기울기 검증
+ 기울기 구하는 방법은 1. 수치 미분 이용 2. 오차 역전파법
+ 수치 미분은 느리지만, 구현이 쉬워서 버그를 찾기가 쉽다.
+ 반대로 오차역전파법은 속도가 빠르나, 구현이 복잡해서 실수가 있을 수 있다.
+ => 수치 미분의 결과와 오차역전파법의 결과를 비교하여, 두 방식으로 구한 기울기가 일치함을 확인하는 작업을 기울기 확인이라 함.

In [33]:
from dataset.mnist import load_mnist

# 데이터 읽기 
(x_train, t_train),(x_test,t_test)=load_mnist(normalize=True, one_hot_label=True)

network=TwoLayerNet(input_size=784, hidden_size=50, output_size=10)
x_batch=x_train[:3]
t_batch=t_train[:3]

grad_numerical=network.numerical_gradient(x_batch, t_batch)
grad_backprop=network.gradient(x_batch, t_batch)

# 각 가중치의 차이의 절댓값을 구한 후, 그 절댓값들의 평균을 낸다.
for key in grad_numerical.keys():
    diff=np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":"+ str(diff))

W1:4.520294807086996e-10
b1:2.639433318324825e-09
W2:5.70191685571409e-09
b2:1.4037041328324172e-07


### 5-7-4. 오차역전파법을 사용한 학습 구현

In [34]:
# 데이터 읽기 
(x_train, t_train),(x_test,t_test)=load_mnist(normalize=True, one_hot_label=True)

network=TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch) # 수치 미분 방식
    grad = network.gradient(x_batch, t_batch) # 오차역전파법 방식(훨씬 빠르다)
    
    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.0795 0.0818
0.90495 0.909
0.9251333333333334 0.9276
0.9385666666666667 0.9396
0.94515 0.946
0.9512666666666667 0.9508
0.95835 0.9549
0.9607833333333333 0.958
0.9659833333333333 0.9616
0.9693833333333334 0.9636
0.9705166666666667 0.9658
0.9729166666666667 0.9657
0.9743833333333334 0.9684
0.976 0.9694
0.9775833333333334 0.9707
0.9790166666666666 0.9714
0.9801 0.9715
